In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D
import pickle
from PIL import Image
from scipy.linalg import eigh
import pickle

Using TensorFlow backend.


In [2]:
path = 'F:\\3-1\\RESIZED_DATASET'
myList = os.listdir(path)
print(len(myList))
noOfClasses = len(myList)
images=[]
classNo = []

38


In [3]:
for x in range(0,noOfClasses):
    myPicList = os.listdir(path+"/"+str(x))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(x)+"/"+y)
        curImg = cv2.resize(curImg,(128,128))
        images.append(curImg)
        classNo.append(x)
    #print(x)

In [4]:
print(len(images))
print(len(classNo))

11061
11061


In [5]:
images = np.array(images)
classNo = np.array(classNo)
print(images.shape)
print(classNo.shape)
testRatio=0.2
validationRatio=0.2

(11061, 128, 128, 3)
(11061,)


# Splitting Data

In [6]:
X_train,X_test,y_train,y_test = train_test_split(images,classNo,test_size=testRatio)
X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=validationRatio)

In [7]:
print(X_train.shape)
print(X_validation.shape)

(7078, 128, 128, 3)
(1770, 128, 128, 3)


In [8]:
numOfSamples= []
for x in range(0,noOfClasses):
    #print(len(np.where(y_train==x)[0]))
    numOfSamples.append(len(np.where(y_train==x)[0]))
print(numOfSamples)

[204, 182, 182, 178, 164, 193, 187, 168, 184, 191, 202, 192, 169, 189, 167, 184, 186, 196, 189, 189, 198, 186, 193, 192, 176, 188, 182, 197, 176, 177, 185, 183, 198, 199, 180, 196, 184, 192]


In [9]:
def preProcessing(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img

In [10]:
X_train = np.array(list(map(preProcessing,X_train)))
X_test = np.array(list(map(preProcessing,X_test)))
X_validation = np.array(list(map(preProcessing,X_validation)))

In [11]:
print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
X_validation = X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)
print(X_train.shape)

(7078, 128, 128)
(7078, 128, 128, 1)


In [12]:
dataGen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10)
dataGen.fit(X_train)

In [13]:
y_train = to_categorical(y_train,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
print(y_train.shape)

(7078, 38)


In [14]:
imageDimensions=(128,128,3)

In [15]:
def myModel():
    noOfFilters = 60
    sizeOfFilter1 = (5,5)
    sizeOfFilter2 = (3, 3)
    sizeOfPool = (2,2)
    noOfNodes= 500
 
    model = Sequential()
    model.add((Conv2D(noOfFilters,sizeOfFilter1,input_shape=(imageDimensions[0],
                      imageDimensions[1],1),activation='relu')))
    model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation='relu')))
    model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(noOfNodes,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses, activation='softmax'))
 
    model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model
 
model = myModel()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 60)      1560      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 60)      90060     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 60)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 30)        16230     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 30)        8130      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 30)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 30)       

In [16]:
batchSizeVal = 50
epochsVal = 10
stepsPerEpochVal = 2000

In [17]:
history=model.fit(X_train, y_train, validation_data=(X_validation,y_validation), epochs=10)

Train on 7078 samples, validate on 1770 samples
Epoch 1/10
7078/7078 [==============================] - 225s 32ms/step - loss: 3.4194 - accuracy: 0.0911 - val_loss: 2.8346 - val_accuracy: 0.2446
Epoch 2/10
7078/7078 [==============================] - 182s 26ms/step - loss: 2.6137 - accuracy: 0.2793 - val_loss: 2.1698 - val_accuracy: 0.3949
Epoch 3/10
7078/7078 [==============================] - 181s 26ms/step - loss: 1.9111 - accuracy: 0.4459 - val_loss: 1.8571 - val_accuracy: 0.4842
Epoch 4/10
7078/7078 [==============================] - 181s 26ms/step - loss: 1.3783 - accuracy: 0.5886 - val_loss: 1.7580 - val_accuracy: 0.5068
Epoch 5/10
7078/7078 [==============================] - 182s 26ms/step - loss: 1.0071 - accuracy: 0.6919 - val_loss: 1.7808 - val_accuracy: 0.5328
Epoch 6/10
7078/7078 [==============================] - 181s 26ms/step - loss: 0.7699 - accuracy: 0.7578 - val_loss: 1.7659 - val_accuracy: 0.5294
Epoch 7/10
7078/7078 [==============================] - 182s 26ms/step

In [19]:
score = model.evaluate(X_test,y_test,verbose=0)
print(score)

[1.8990095019394382, 0.5666515827178955]


In [20]:
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
while True:
    success,imgOriginal = cap.read()
    img = np.asarray(imgOriginal)
    img = cv2.resize(img,(128,128))
    img = preProcessing(img)
    cv2.imshow("Processed Image",img)
    img=img.reshape(1,128,128,1)
    classIndex = int(model.predict_classes(img))
    #print(classIndex)
    predictions = model.predict(img)
    probVal = np.amax(predictions)
    cv2.putText(imgOriginal,str(classIndex)+" "+str(probVal),(50,50),
               cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
    cv2.imshow("Original Image",imgOriginal)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [26]:
newpath='F:\\3-1\\RESIZED_TESTING_DATA\\8\\20180714_232511.jpg'
image = cv2.imread(newpath)
cv2.imshow("Processed Image",image)

In [41]:
newpath='F:\\3-1\\RESIZED_TESTING_DATA\\20\\20180716_000301.jpg'
while True:
    #success,imgOriginal = cap.read()
    imgOriginal = cv2.imread(newpath)
    img = cv2.imread(newpath)
    #img = np.asarray(imgOriginal)
    img = cv2.resize(img,(128,128))
    img = preProcessing(img)
    cv2.imshow("Processed Image",img)
    img=img.reshape(1,128,128,1)
    classIndex = int(model.predict_classes(img))
    #print(classIndex)
    predictions = model.predict(img)
    probVal = np.amax(predictions)
    cv2.putText(imgOriginal,str(classIndex)+" "+str(probVal),(50,50),
               cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
    cv2.imshow("Original Image",imgOriginal)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()